In [30]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, adam
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
# read in data
data_train = pd.read_csv('train.csv')
data_val = pd.read_csv('test.csv')

names_train = data_train['Name']
names_val = data_val['Name']
# drop columns that do not correlate with survival / have significant gaps
data_train = data_train.drop(['Cabin', 'Name', 'PassengerId', 'Fare', 'Embarked'], axis=1)
data_val = data_val.drop(['Cabin', 'Name', 'PassengerId', 'Fare', 'Embarked'], axis=1)
data = [data_train, data_val]
classes = ['Deceased', 'Survived']

In [31]:
data

[     Survived  Pclass     Sex   Age  SibSp  Parch            Ticket
 0           0       3    male  22.0      1      0         A/5 21171
 1           1       1  female  38.0      1      0          PC 17599
 2           1       3  female  26.0      0      0  STON/O2. 3101282
 3           1       1  female  35.0      1      0            113803
 4           0       3    male  35.0      0      0            373450
 5           0       3    male   NaN      0      0            330877
 6           0       1    male  54.0      0      0             17463
 7           0       3    male   2.0      3      1            349909
 8           1       3  female  27.0      0      2            347742
 9           1       2  female  14.0      1      0            237736
 10          1       3  female   4.0      1      1           PP 9549
 11          1       1  female  58.0      0      0            113783
 12          0       3    male  20.0      0      0         A/5. 2151
 13          0       3    male  39

In [32]:
def vec_to_description(person):
	description = ""
	alone = None
	ticket = None
	if person[0] == 1:
		sex = "Male"
	else:
		sex = "Female"
	if person[1] == 1:
		alone = "with large family"
	if person[2] == 1:
		alone = "alone"
	if person[3] == 1:
		age = "Child"
	if person[4] == 1:
		age = "Young adult"
	if person[5] == 1:
		age =  "Adult"
	if person[6] == 1:
		age = "Elderly"
	if person[7] == 1:
		ticket = "travelling in first class"
	if person[8] == 1:
		ticket = "travelling in second class"
	if person[9] == 1:
		ticket = "travelling in third class"

	if(ticket is None):
		ticket = "travelling in UNKNOWN"
	if(alone is None):
		alone = "with companion / or family with 4 members or less"
	description = ("{0}, {1}, {2} {3}").format(age, sex, ticket, alone)

	return description

In [33]:
for d in data:
	'''
	Preprocessing
	1. Map columns containing strings to binary numerical values
	   1 = male, 0 = female
	2. Identify and flag large families, families with 5 or more
	   members had significantly lower survival rates
	3. Identify and flag solo travellers, they had lower survival rates
	4. If age is not known for a person then populate with median
	5. Create age groups for child, young adult, adult and old
	6. Group by ticket class
	'''
	# Create new columns for relevant groups
	d['largeFam'] = 0
	d['Alone'] = 0
	d['Child'] = 0
	d['YoungA'] = 0
	d['Adult'] = 0
	d['Old'] = 0
	d['First_class'] = 0
	d['Second_class'] = 0
	d['Third_class'] =0

	# 1. Convert sex to  binary numerical values
	d['Sex'] = d['Sex'].map({ 'male': 1, 'female': 0}).astype(int)

	# 2. Identify + flag large groups/families
	d.loc[d['SibSp'] + d['Parch'] + 1 > 4, 'largeFam'] = 1
	
	# 3. Identify + flag solo travellers
	d.loc[d['SibSp'] + d['Parch'] + 1 == 1, 'Alone'] =1

	# 4. Set blank ages to median age
	d.loc[d.Age.isnull(), 'Age'] = d.Age.median()
	
	# 5. Group passengers by age
	d.loc[d['Age'] <= 16, 'Child'] = 1
	d.loc[(d['Age'] > 16) & (d['Age'] <= 30), 'YoungA'] = 1
	d.loc[(d['Age'] > 30) & (d['Age'] <= 60), 'Adult'] = 1
	d.loc[d['Age'] > 60, 'Old'] = 1

	#6. Group passengers by ticket class
	d.loc[d['Pclass'] == 1, 'First_class'] = 1
	d.loc[d['Pclass'] == 2, 'Second_class'] = 1
	d.loc[d['Pclass'] == 3, 'Third_class'] = 1

In [34]:
# Drop columns we don't need any more from training/validation set
# We've moved these into groups above
data_train = data_train.drop(['Ticket', 'Parch', 'SibSp', 'Age', 'Pclass'], axis=1)
data_val = data_val.drop(['Ticket', 'Parch', 'SibSp', 'Age', 'Pclass'], axis=1)

# Set target column to predict (Survived)
targets = np.array(to_categorical(data_train['Survived'], 2))

# Set features we are trying to correlate to target
features = np.array(data_train.drop('Survived', axis=1))
features_val = np.array(data_val)

In [35]:
# Set neural net architecture
# 2 simple fully connected layers with relu activation
# output layer will have 2 nodes (len classes) with softmax activation
# to provide probability score
model = Sequential()
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(Dense(len(classes), activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])


model.fit(features, targets, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
891/891 [==============================] - 0s 383us/step - loss: 0.5814 - acc: 0.7363
Epoch 2/100
891/891 [==============================] - 0s 71us/step - loss: 0.5041 - acc: 0.7868
Epoch 3/100
891/891 [==============================] - 0s 71us/step - loss: 0.4792 - acc: 0.8013
Epoch 4/100
891/891 [==============================] - 0s 73us/step - loss: 0.5030 - acc: 0.7811
Epoch 5/100
891/891 [==============================] - 0s 71us/step - loss: 0.4777 - acc: 0.8002
Epoch 6/100
891/891 [==============================] - 0s 74us/step - loss: 0.4700 - acc: 0.8036
Epoch 7/100
891/891 [==============================] - 0s 71us/step - loss: 0.4837 - acc: 0.7946
Epoch 8/100
891/891 [==============================] - 0s 72us/step - loss: 0.4684 - acc: 0.7901
Epoch 9/100
891/891 [==============================] - 0s 72us/step - loss: 0.4538 - acc: 0.8114
Epoch 10/100
891/891 [==============================] - 0s 77us/step - loss: 0.4619 - acc: 0.7901
Epoch 11/100
891/891 [======

891/891 [==============================] - 0s 71us/step - loss: 0.4079 - acc: 0.8272
Epoch 85/100
891/891 [==============================] - 0s 67us/step - loss: 0.4195 - acc: 0.8238
Epoch 86/100
891/891 [==============================] - 0s 66us/step - loss: 0.4088 - acc: 0.8182
Epoch 87/100
891/891 [==============================] - 0s 69us/step - loss: 0.4149 - acc: 0.8249
Epoch 88/100
891/891 [==============================] - 0s 88us/step - loss: 0.4089 - acc: 0.8215
Epoch 89/100
891/891 [==============================] - 0s 88us/step - loss: 0.4113 - acc: 0.8249
Epoch 90/100
891/891 [==============================] - 0s 81us/step - loss: 0.4099 - acc: 0.8272
Epoch 91/100
891/891 [==============================] - 0s 67us/step - loss: 0.3934 - acc: 0.8204
Epoch 92/100
891/891 [==============================] - 0s 66us/step - loss: 0.4093 - acc: 0.8215
Epoch 93/100
891/891 [==============================] - 0s 66us/step - loss: 0.4097 - acc: 0.8305
Epoch 94/100
891/891 [===========

In [36]:
p = model.predict(features_val)
for i, x in enumerate(p):
	n = names_val[i]
	d = features_val[i]
	d = vec_to_description(d)
	mortality = classes[np.argmax(x)]
	if (mortality == 'Deceased'):
		prob = x[0]
	else:
		prob = x[1]
	
	print("\n\n\n\n{0}\n________________________________________\nDescription: {1}".format(n, d)) 
	print("Prediction: {0} ({1:.1%} probability)".format(mortality, prob))





Kelly, Mr. James
________________________________________
Description: Adult, Male, travelling in third class alone
Prediction: Deceased (81.3% probability)




Wilkes, Mrs. James (Ellen Needs)
________________________________________
Description: Adult, Female, travelling in third class with companion / or family with 4 members or less
Prediction: Deceased (62.0% probability)




Myles, Mr. Thomas Francis
________________________________________
Description: Elderly, Male, travelling in second class alone
Prediction: Deceased (76.4% probability)




Wirz, Mr. Albert
________________________________________
Description: Young adult, Male, travelling in third class alone
Prediction: Deceased (88.7% probability)




Hirvonen, Mrs. Alexander (Helga E Lindqvist)
________________________________________
Description: Young adult, Female, travelling in third class with companion / or family with 4 members or less
Prediction: Survived (53.5% probability)




Svensson, Mr. Johan Cervin
____